In [ ]:
# verigy whether nonflat/distort boundary models are easier to be adversarial attack
# time or distortion
# here we choose time

In [ ]:
# fgsm attack
# import modules
from deepvisualinsight.MMS import MMS
from deepvisualinsight import utils
import sys
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import json
import tensorflow as tf
import torchvision.transforms as transforms
import torchvision

In [ ]:
content_path = "E:\\DVI_exp_data\\active_learning\\LeastConfidence"
sys.path.append(content_path)

from Model.model import *
net = ResNet18()
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

mms = MMS(content_path, net, 1, 20, 1, 512, 10, classes, cmap="tab10", resolution=100, neurons=256,
          verbose=1, temporal=False, split=-1, advance_border_gen=True, attack_device="cuda:0")

In [ ]:
# hyperparameters
EPOCH=10

In [ ]:
model_location = os.path.join(mms.model_path, "Epoch_{:d}".format(EPOCH), "subject_model.pth")
mms.model.load_state_dict(torch.load(model_location, map_location=torch.device("cpu")))
mms.model = mms.model.to(mms.device)
mms.model.eval()

In [ ]:
criterion = nn.CrossEntropyLoss()
softmax = torch.nn.Softmax(dim=-1)
epsilons = [.01,.03,.05,.1]
epsilon = .01

def adv_attack(image, epsilon, data_grad):

    # Collect the element-wise sign of the data gradient
    sign_data_grad = torch.sign(data_grad)

    perturbed_image = image + epsilon*sign_data_grad

    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image

In [ ]:
# fgsm: how many adv samples we can get
# ifgsm: set a time limit
TEST_BUDGET = 1000

CIFAR_NORM = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(*CIFAR_NORM)])
testset = torchvision.datasets.CIFAR10(root='data', train=False,
                                       download=True, transform=transform)
test_subset = torch.utils.data.Subset(testset, np.arange(TEST_BUDGET))
testloader = torch.utils.data.DataLoader(test_subset, batch_size=1,
                                         shuffle=False, num_workers=2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("attack with epsilon {}...".format(epsilon))

succ = 0
for i, (data, target) in enumerate(testloader, 0):
    # Send the data and label to the device
    data, target = data.to(device), target.to(device)
    # Set requires_grad attribute of tensor. Important for Attack
    data.requires_grad = True
    # Forward pass the data through the model
    output = mms.model(data)
    init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

    # If the initial prediction is wrong, dont bother attacking, just move on
    if init_pred.item() != target.item():
        continue

    # Calculate the loss
    loss = torch.nn.functional.nll_loss(output, target)

    # Zero all existing gradients
    mms.model.zero_grad()

    # Calculate gradients of model in backward pass
    loss.backward()

    # Collect datagrad
    data_grad = data.grad.data

    # Call FGSM Attack
    perturbed_data = adv_attack(data, epsilon, data_grad)

    # Re-classify the perturbed image
    output = mms.model(perturbed_data)

    # Check for success
    final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
    if final_pred.item() != target.item():
        succ += 1
succ